In [45]:
from pandas import read_excel
from pandas import DataFrame
from pandas import concat
from pandas import Series
import numpy
from numpy import append
from numpy import array
from numpy import diff
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from math import sqrt
import warnings
warnings.filterwarnings("ignore")
import sage

In [46]:
# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return array(diff)

In [47]:
# scale train and test data to [-1, 1]
def scale(train, test):
    # fit scaler
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(train)
    # transform train
    train = train.reshape(train.shape[0], train.shape[1])
    train_scaled = scaler.transform(train)
    # transform test
    test = test.reshape(test.shape[0], test.shape[1])
    test_scaled = scaler.transform(test)
    return scaler, train_scaled, test_scaled

In [48]:
def get_technical_indicators(data):
    # Create Moving Average
    data = DataFrame(data)
    dataset = DataFrame()
    dataset['ma5'] = data.rolling(window=5).mean()
    dataset['ma20'] = data.rolling(window=25).mean()
    
    # Create MACD
    dataset['25ema'] = data.ewm(span=25).mean()
    dataset['10ema'] = data.ewm(span=10).mean()
    dataset['MACD'] = dataset['10ema']-dataset['25ema']

    # Create Bollinger Bands
    dataset['20sd'] = data.rolling(window = 20).std()
    dataset['upper_band'] = dataset['ma20'] + (dataset['20sd']*2)
    dataset['lower_band'] = dataset['ma20'] - (dataset['20sd']*2)
    
    # Create Exponential moving average
    dataset['ema'] = data.ewm(com=0.5).mean()
    
    # Create Momentum
    dataset['momentum'] = data-1
    # dataset['log_momentum'] = log(dataset['momentum'])
    return dataset.dropna()

In [49]:
def timeseries_to_supervised(data, lag):
    df = DataFrame(data)
    columns = [df.shift(i) for i in range(1, lag+1)]
    columns.append(df)
    df = concat(columns, axis=1)
    return DataFrame(df)

In [50]:
# fit ML model
def model_fit(X,y):
    params = {'max_depth': range(3,21,3),
              'min_child_weight': range(1,5,1),
              'learning_rate': [0.001,0.01,0.1],
              'n_estimators': [250,500,750,1000],
              'colsample_bytree': [i/10.0 for i in range(3,10)],
              'colsample_bylevel': [i/10.0 for i in range(3,10)],
              'gamma': [i/10.0 for i in range(0,5)]}
    model = RandomizedSearchCV(XGBRegressor(),params,cv=5,n_iter=20,refit="AUC",verbose=1,n_jobs=-1)
    model_fit = model.fit(X,y)
    return model_fit

In [51]:
def experiment(series, features):
    # transform data to be stationary
    raw_values = series.values
    diff_values = difference(raw_values)
    
    # split data into train and test-sets
    len_test = 150
    train = diff_values[:-len_test]
    test = diff_values[-len_test:]
    
    # transform the scale of the data
    scaler, train_scaled, test_scaled = scale(train, test)
    
    # get technical indicators
    df = get_technical_indicators(train_scaled)
    
    # transform data to be supervised learning
    supervised = timeseries_to_supervised(train_scaled, features)
    supervised_values = supervised.values[features:,:]
    
    new_df = concat([df, DataFrame(supervised_values)],axis=1).dropna()
    
    # fit the base model
    X,y = new_df.iloc[:,:-1], new_df.iloc[:,-1]
    mod = model_fit(X,y)
    
    # Calculate SAGE values
    imputer = sage.MarginalImputer(mod.predict, X.values)
    estimator = sage.PermutationEstimator(imputer, 'mse')
    sage_values = estimator(X.values,y.values)
    
    return sage_values.values

In [52]:
path = "C:\\Users\\fatim\\Box\\Habbab Fatima\\Experiment set 3\\DataSets\\"
asset_list = read_excel("C:\\Users\\fatim\\Box\\Habbab Fatima\\Experiment set 6\\asset_list.xlsx")

no_assets = len(asset_list)

sage_data = DataFrame()

for i in range(0,no_assets):
    ticker = asset_list.iloc[i,0]
    data = read_excel(path+ticker+".xlsx")
    series = data[["Close"]].dropna()
    
    no_features = 5
    
    sage_results = experiment(series, no_features)

    sage_data[ticker] = sage_results

Fitting 5 folds for each of 20 candidates, totalling 100 fits


  0%|          | 0/1 [00:00<?, ?it/s]

Fitting 5 folds for each of 20 candidates, totalling 100 fits


  0%|          | 0/1 [00:00<?, ?it/s]

Fitting 5 folds for each of 20 candidates, totalling 100 fits


...

Fitting 5 folds for each of 20 candidates, totalling 100 fits


  0%|          | 0/1 [00:00<?, ?it/s]

Fitting 5 folds for each of 20 candidates, totalling 100 fits


  0%|          | 0/1 [00:00<?, ?it/s]

ValueError: cannot convert float NaN to integer

In [54]:
sage_data.to_excel("sage_values.xlsx")